# Extra examples

Yeah, I can't think of any names.

In [31]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from samplics.categorical import CrossTabulation

gss_saf = pd.read_csv("data/safiya_clean.csv")
gss_theo = pd.read_csv("data/theo_clean.csv")

## Additional cross tabs

In [32]:

tab_imm_race = CrossTabulation("proportion")
tab_imm_race.tabulate(
    vars=gss_saf[["immigration_cat", "sex_cat"]],
    samp_weight=gss_saf["wtssnrps"],
    # stratum=gss_saf["vstrat"],
    psu=gss_saf["vpsu"],
    remove_nan=True
)

tab_imm_race.to_dataframe()

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 3 is different from 4)

In [2]:
gss_saf

,year,age,income,education_cat,president_cat,sex_cat,race_cat,party_cat,language_cat,attitude_cat,immigration_cat,vpsu,vstrat,wtssnrps
0,2010,31.0,66239.250,College,Obama,Male,Other,Democrat,Yes,Remained the same,Stay same and Increase,1,2240,1.083753
1,2010,23.0,13047.125,College,Obama,Female,White,Democrat,No,Increased a little,Stay same and Increase,1,2240,0.456877
2,2010,71.0,17061.625,High school or less,Obama,Female,Black,Democrat,Yes,Reduced a little,Reduce,2,2240,0.462404
3,2010,82.0,7226.100,High school or less,Obama,Female,White,Republican,No,Remained the same,Stay same and Increase,2,2240,0.557622
4,2010,78.0,NaN,No schooling,Obama,Female,Black,Independent,No,NaN,NaN,1,2241,1.286242
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15798,2021,21.0,6048.000,College,Trump,Female,White,Democrat,Yes,Increased a little,Stay same and Increase,1502,2655,3.585827
15799,2021,29.0,80640.000,College,Trump,Female,White,Republican,No,Reduced a little,Reduce,4056,2657,0.833348
15800,2021,NaN,168736.297,College,Trump,Female,Other,Democrat,Yes,Remained the same,Stay same and Increase,1705,2659,1.042119
15801,2021,68.0,1344.000,Ph. D/Doctorate,Trump,Female,White,Democrat,Yes,Increased a little,Stay same and Increase,3192,2652,0.637692


In [30]:
gss_theo = pd.read_csv("data/theo_clean.csv")

gss_theo.astype({"sex": "category", "race": "category", "region": "category", "ethnic": "category"})
gss_theo


,coninc,coninc_bins,income,region,sex,race,talkspvs,ethnic,wtssall,vstrat,vpsu
0,66239.250,"(44930.116, 89524.232]",<$8K,Middle Atlantic,Male,Other,NaN,NaN,0.437745,2106,1
1,13047.125,"(157.623, 44930.116]",$15K+,Middle Atlantic,Male,Other,NaN,NaN,0.437745,2106,1
2,17061.625,"(157.623, 44930.116]",$15K+,Middle Atlantic,Male,Black,NaN,Africa,2.135340,2106,1
3,7226.100,"(157.623, 44930.116]",$15K+,Middle Atlantic,Male,Black,NaN,Africa,1.313234,2106,1
4,NaN,NaN,$15K+,Middle Atlantic,Female,Black,NaN,NaN,0.437745,2106,2
...,...,...,...,...,...,...,...,...,...,...,...
17821,NaN,NaN,$8K to $15K,NaN,Female,NaN,NaN,NaN,NaN,2655,1502
17822,NaN,NaN,$15K+,NaN,Female,NaN,NaN,NaN,NaN,2657,4056
17823,NaN,NaN,NaN,NaN,Female,NaN,NaN,NaN,NaN,2659,1705
17824,NaN,NaN,<$8K,NaN,Female,NaN,NaN,NaN,NaN,2652,3192


## Plots